### Regresion Lineal Múltiple Regresión con validacion cruzada 
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression

Métricas: https://scikit-learn.org/stable/api/sklearn.metrics.html

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
#Automcompletar rápido
%config IPCompleter.greedy=True
#Cargamos el dataset sintetico_binaria.csv que está en 00_DATASETS y seleccionamos 5 variables para que el modelo funcione bien
df = pd.read_csv('../03_MACHINE_LEARNING/00_DATASETS/sintetico_continua.csv')
df = df[['x3','x5','x1','x8','x4','target']]
df.head(3)

,x3,x5,x1,x8,x4,target
0,1.733067,0.047734,-0.333643,-0.310969,0.457594,137.101716
1,0.962427,0.538592,-0.915244,-1.306736,0.285146,43.190649
2,-1.999640,0.826154,0.719608,0.094109,-0.893353,-78.126159


In [3]:
#1. Preparamos los datasets: x e y
x = df.drop(columns='target')
y = df['target']

#2. Dividimos los datasets en 4 objetos y le ponemos Validación al grupo de Test
from sklearn.model_selection import train_test_split
train_x, val_x, train_y, val_y = train_test_split(x, y, test_size = 0.3)

#3. Instanciamos el modelo y el objeto de validación cruzada para comprobar la estabilidad del modelo sobre el grupo de Train
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
rm = LinearRegression()
kf = KFold(n_splits = 5)

#4. Entrenamos los modelos y evaluamos
from sklearn.model_selection import cross_val_score
metricas_cv_rm = cross_val_score(estimator = rm,
                                 X = train_x,
                                 y = train_y,
                                 cv = kf,
                                 scoring = 'neg_mean_absolute_error')
print('Métricas RM Train: ', metricas_cv_rm)             #Comprobamos que los resultados de la validación son estables
print('Media métricas RM Train: ', metricas_cv_rm.mean())#o iteramos haciendo cambios en las variables

#5. Modelizamos sobre train con el modelo
train_rm_fit = rm.fit(train_x,np.ravel(train_y))#El ravel es para pasarlo en formato plano y que no salga warning

#6. Modelizamos y predecimos sobre validación con el modelo
val_rm_fit = rm.fit(val_x.values,np.ravel(val_y))#np.ravel es para pasarlo en formato plano y que no salga warning
val_rm_sco = val_rm_fit.predict(val_x.values)

#Extraemos la métrica de medicición del error sobre el grupo de validación
from sklearn.metrics import mean_absolute_error
print('Métricas RM Validación: ', mean_absolute_error(val_y, val_rm_sco))#Regresión Múltiple da 4.9 por lo que predice mejor

Métricas RM Train:  [-5.00390105 -4.75688308 -4.7158381  -5.08425794 -4.71443107]
Media métricas RM Train:  -4.855062248263171
Métricas RM Validación:  4.907544944538101


In [4]:
pd.DataFrame({'real':val_y,'prediccion':val_rm_sco}).head(2)

,real,prediccion
244,-281.923885,-279.141971
31,-290.319639,-292.640190
